# Assignment 3
- **Assigned:** Friday, March 4
- **Due:** Wednesday, March 16 @ 5pm

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
#from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import gridspec
from matplotlib.cm import get_cmap

import numpy as np
from numpy.random import randn
from numpy import eye, array, asarray, exp

from math import sqrt
from scipy.linalg import expm
from scipy.linalg import solve_discrete_lyapunov
from scipy.linalg import sqrtm
from scipy import linalg as la
from scipy.integrate import odeint

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets 
from IPython.display import display

float_formatter = "{:.4f}".format
np.set_printoptions(formatter={'float': '{: 0.4f}'.format})
np.set_printoptions(precision=3)
plt.rcParams["font.serif"] = "cmr12"
plt.rcParams['savefig.dpi'] = 300
plt.rcParams["figure.dpi"] = 100

# Homework


$
\newcommand{\ux}{\underline x}
\newcommand{\uy}{\underline y}
\newcommand{\uv}{\underline v}
\newcommand{\hx}{\hat {\underline x}}
\newcommand{\hxm}{\hat {\underline x}(-)}
\newcommand{\hxp}{\hat {\underline x}(+)}
\newcommand{\jpssm}{Q^{\rm ss}_{k|k-1}}
\newcommand{\jpssp}{Q^{\rm ss}_{k|k}}
$

## Problem 1 -- Discrete Steady State

Calculate the steady
state error covariance matrices $\jpssm$ and $\jpssp$ for the
system:
\begin{eqnarray}
x_{k+1} &=& \alpha x_k + w_k \\ y_k &=& 3 x_k + v_k
\end{eqnarray}
where the Gaussian white noises $w_k$ and $v_k$ are uncorrelated with
the state $x$ (and each other), have zero mean, and covariances $W_d$ and $V$,
respectively.  
(a) Solve the resulting quadratic equation to determine explicit
expressions for $\jpssm$ and $\jpssp$.
* Express your solution in terms of $\alpha,$ $W_d$, and
$\gamma = V/W_d$ which is the relative magnitude of the noises.
* The constraint that the covariance matrix is at least
positive semi-definite (PSD) should be used to select between the
two possible solutions of the quadratic equation.
* Is a solution possible for all values of $\alpha$?

(b) Use the expressions in part (a) to calculate the limits of
$\jpssp$ as $\gamma \rightarrow 0$ and $\gamma \rightarrow \infty$
for the two cases $\alpha < 1$ and $\alpha > 1$.
* What do these results imply about our ability to estimate
the states of stable (or unstable) systems with relatively clean
(or noisy) measurements?

(c) If $\alpha=0.975$, $W_d=1$, $R=100$, use the recursive and Riccati equation solvers in Python to confirm that your steady state predictions for  $\jpssm$ and $\jpssp$ are accurate. Confirm for the unstable case $\alpha=1.1$ too.


$\newcommand{\bmat}{\begin{bmatrix}}
\newcommand{\emat}{\end{bmatrix}}
\newcommand{\expect}[1]{\expec\left[ #1 \right]}
\newcommand{\mb}{\mathbf}
\newcommand{\argmax}{\operatornamewithlimits{argmax}}
\newcommand{\argmin}{\operatornamewithlimits{argmin}}
\newcommand{\bea}{\begin{eqnarray}}
\newcommand{\eea}{\end{eqnarray}}
\newcommand{\beas}{\begin{eqnarray*}}
\newcommand{\eeas}{\end{eqnarray*}}
\newcommand{\ba}{\begin{array}}
\newcommand{\ea}{\end{array}}
\newcommand{\njbv}{\mathbf{v}}
\newcommand{\njbw}{\mathbf{w}}
\newcommand{\njbx}{\mathbf{x}}
\newcommand{\njby}{\mathbf{y}}
$

## Problem 2 -- Steady State Covariance 

There are not many steady state covariance problems that can be solved analytically. This is one of them. Consider the second order system with
dynamics
\begin{eqnarray}
\dot {\njbx}(t) &=& \left[ \ba{cc} 0 & 1\\-\omega^2 & -2\zeta\omega \ea
\right] \njbx(t) + \left[ \ba{c} 0 \\  \omega \ea \right]w(t)\\
\njby(t) &=& C \njbx(t) + \njbv(t)
\end{eqnarray}
where $w(t) \sim N(0,R_w)$ and $v(t)$ are uncorrelated, Gaussian white noises. Here, $\omega>0$ and $\xi>0$. 

**Part 1**.- Solve for the analytic expressions of the steady state covariance matrix $Q_\infty$ for two different cases:
*   case 1: $C=[\ba{cc} \omega, & 0 \ea]$ and $v \sim N(0,R_{v1})$
*   case 2: $C=[\ba{cc}0, & 1 \ea]$ and $v \sim N(0,R_{v2})$

> **Hint**: You will end up with a system of quadratic equations. To choose the sign in the solution of quadratic equations, remember that the covariance matrix must be a positive semi-definite (PSD) matrix

> **Note**: You can solve the quadratic equations by hand, but you can also use a symbolic solver (SymPy/Matlab/Mathematica/...) if you prefer.

**Part 2**.-  Use the results from cases 1 and 2 to analyze the impact of sensor
selection on the estimate uncertainty (i.e., look at the magnitudes of
$Q_\infty(1,1)$ and $Q_\infty(2,2)$). What impact does this selection
have on the dynamics of the estimate error? To simplify the expressions a bit, consider the case where $\zeta \ll 1$ and $\frac{R_w}{R_{v}} \gg 1$. 

**Part 3**.-  Confirm your results with Python for $\omega=1.5$, $\zeta=0.02$,
$R_w=2$, $R_{v1}=0.25$ and $R_{v2}=0.04$.

Note that even slight deviations from this problem setting lead to much harder Riccati equations to solve!

## Problem 3 -- Basic Optimal Estimation

Consider the case of estimation a static, scalar variable $x$. We take two noisy measurements of the scalar $x$ of the form:

$$
z_i = h_i x + \theta_i,\qquad i=1,2
$$

where $h_1=1$ and $h_2=2$. Assume that the noise terms $\theta_i$ are uncorrelated, and we have statistics $E[\theta_i]=0$ and $E[\theta_i\cdot\theta_i]=\sigma_i^2$. Also assume that $E[x\cdot\theta_i]=0$, but little else is known about $x$.

1. Assume an optimal estimate of the form

    $$
    \hat{x} = l_1 z_1 + l_2 z_2
    $$

    where the $l_i$ are constants that are statistically independent of $x$. Compute the values of $l_i$ that
    
    (i) ensures that the estimate $\hat{x}$ is unbiased, <br>
    (ii) minimizes the mean-square estimation error $E[(x-\hat{x})^2]$.
    
    Use these gains to compute the minimum value of the mean-square estimation error. Perform the scalar minimization with a simple optimization with respect to the estimator gains from first principles.


2. Discuss how the optimal estimator uses the two measurements for the following cases:
* $\sigma_2 \gg \sigma_1$ <br>
* $\sigma_2 = \sigma_1$ <br>
* $\sigma_2 \ll \sigma_1$

Do these special cases make sense and agree with your intuition? Explain.
    
    
3. Determine the WLSE discussed in class for this problem. Make sure that you pick the weighting matrix appropriately. Compare the gains of the WLSE with those from (1) and (2).    

## Problem 4 -- Gyroscope Model
$\newcommand{\expec}{{\Bbb E}}$
$\newcommand{\expect}[1]{\expec\left[ #1 \right]}$
$\newcommand{\mb}{\mathbf}$

As explained in Lecture 10 (slide 10-22), a simple model of the long term drift
of a gyroscope is given by the output $d(t)$ of the linear system

$$
G_d(s) = \frac{d}{w}=\frac{\alpha}{s+\alpha}
$$

where $w(t)$ is Gaussian white noise with
$ \expec[w(t)]=0, $
$\expec[w(t)w(t+\tau)]=W_c \delta(\tau),$ and $W_c=2 {\rm deg}^2/{\rm hr}
$.

It is initially assumed that noisy measurements of the Gyro drift rate
are available every 0.25 hr

$$
z(t_i) = d(t_i) + v(t_i)
$$
where $v(t_i)$ is discrete white Gaussian noise with

$$
\expec[v(t_i)]=0, \quad \expec[v(t_i)v(t_j)]=R \Delta_{ij}, \quad
R=0.5 {\rm deg}^2/{\rm hr}^2
$$

1) Develop the continuous and discrete equations of motion for this system using a shaping filter break frequency of 1 rad/hr (i.e., $\alpha=1$).

2) Use $\hat x(t_0)=0$ and $Q(t_0)=1$ deg$^2$/hr$^2$ and calculate $Q_i(-)$ and $Q_i(+)$ for $i=0,1,2,3,4$. Confirm that at this point in the iteration ($i=4$) the covariance values 
have essentially converged to their steady state values.
Plot the discrete $Q_i$ versus $i$ for this system with discrete measurements. 

> **Hint:** Compare the plot and values you obtain with the ones shown on slide 10-22.

3) It turns out that a continuous measure of the gyro drift is also available, with
$$
z_c(t) = d(t) + v_c(t)
$$
where $v_c(t)$ is a continuous white Gaussian noise with
$$
\expec[v_c(t)]=0,\quad \expec[v_c(t)v_c(t+\tau)]=R_c(t) \delta(\tau)
$$
To maintain consistency with the discrete measurements, we set
$$
R_c(t)=R_c=[R \Delta t] = (0.5 {\rm deg}^2/{\rm hr}^2)(0.25 {\rm hr}) = 0.125 {\rm deg}^2/{\rm hr}
$$
Show that for this problem with $t_0=0$ the error covariance of the continuous system satisfies the equation
$$
Q(t) = \frac{aQ_0 + (\alpha^2W_c-\alpha Q_0)\tanh(at)}{a + \left(\alpha + \frac{Q_0}{R_c}\right)\tanh(at)}
$$
where 
$$
a^2 = \alpha^2 + \alpha^2\frac{W_c}{R_c}
$$
There are some extra notes on canvas about how to solve Riccati equations.

4) Plot this function on the same graph as the discrete covariance and comment on the differences between the two curves.


# Lab

## Estimating Tilt

An IMU has x-y-z accelerometers (measuring linear acceleration) and x-y-z gyroscopes (measuring angular velocity). Using these measurements, we would like to measure the tilt angle of an IMU. Here, tilt refers to the roll (rotation about the x axis) and pitch (rotation about the y axis). Note that the yaw (rotation about the z axis) bias is unobservable when only given a gyro and an accelerometer, as we will discuss below.

A naive solution for recovering tilt angle is to integrate the gyroscope measurements over time. However, noise and biases in the gyro measuring process would quickly cause this integration to diverge. Alternatively, accelerometer measurements can provide a measure of the angles, but these measurements are noisy and become distorted for  an accelerated body. By *fusing* these measurement sources together in a Kalman filter, we can leverage the best of both measurement sources.

Note that for simplicity, our solution will only be valid for small tilt angles, less than $\pm90$ deg in each axis.

### Accelerometers as Inclination Sensors

In the previous lab, we studied the measurement model of an accelerometer. The values $\left[\begin{array}{c}
a_{x}\\
a_{y}\\
a_{z}
\end{array}\right]$ reported by an accelerometer can be used to measure the static tilt angle (about x and y). Here, *static* emphasizes that the tilt angle is only valid when the accelerometer is not accelerating, i.e., the force of gravity is the only external force acting on the sensor. Using trigonometry, the tilt angles about x and y can be recovered given the accelerometer measurement (see [here](https://www.digikey.com/en/articles/using-an-accelerometer-for-inclination-sensing) for figures, equations, and discussions). Specifically, for this lab you will use this formula:

$$
\left[\begin{array}{c}
y_{\phi}\\
y_{\theta}
\end{array}\right]=\left[\begin{array}{c}
\text{atan}\left(\frac{a_{y}}{\sqrt{\left(a_{x}^{2}+a_{z}^{2}\right)}}\right)\\
\text{atan}\left(\frac{-a_{x}}{\sqrt{\left(a_{y}^{2}+a_{z}^{2}\right)}}\right)
\end{array}\right]=\left[\begin{array}{c}
\phi\\
\theta
\end{array}\right]+\mathbf{w}
$$

where $\phi$ is roll and $\theta$ is pitch, and $\mathbf{w}$ represents the noise.

### Gyroscopes

The angular rate $\boldsymbol{\omega}=\begin{bmatrix}p & q & r\end{bmatrix}^\top$ of the body of the IMU with respect to the inertial frame, expressed in the body frame, is measured by the rate gyro as

$$
\underbrace{\left[\begin{array}{c}
y_{p}\\
y_{q}\\
y_{r}
\end{array}\right]}_{:=\mathbf{y}_{\boldsymbol{\omega}}}=\underbrace{\left[\begin{array}{c}
p\\
q\\
r
\end{array}\right]}_{:=\boldsymbol{\omega}}+\underbrace{\left[\begin{array}{c}
b_{p}\\
b_{q}\\
b_{r}
\end{array}\right]}_{:=\mathbf{b}_{\boldsymbol{\omega}}}+\boldsymbol{\nu}
$$

where $\boldsymbol{\nu}\sim\mathcal{N}(0,\Sigma_\boldsymbol{\omega})$ is the noise introduced in the measurement process and $\mathbf{b}_\boldsymbol{\omega}$ is a constant or slowly time-varying bias (i.e., $\dot{\mathbf{b}}_\boldsymbol{\omega}\approx0$).

Gyros typically have a sample rate of 500 Hz, 1 kHz, or 8 kHz.
They are great at capturing high-frequency dynamics and are incredibly useful, especially over short time windows.
However, due to the low-frequency bias $\mathbf{b}_\boldsymbol{\omega}$, simply integrating gyros over long timescales is unwise (e.g., more than tens of seconds).

### Attitude Kinematics

To develop a state dynamics equation, we need to understand how the angular rates and the tilt angles are related. The attitude of a rigid body evolves according to
$
\dot{R} = R\omega_\times
$, where $R\in\text{SO}(3)$ is a $3\times 3$ matrix and $\omega_\times$ denotes a [skew-symmetric matrix](https://en.wikipedia.org/wiki/Skew-symmetric_matrix#Cross_product). This is the preferred form of integrating angular velocities and levarages the geometry of $\text{SO}(3)$, but adds complications for our simple tilt estimator. Instead, we will use [Euler angles](https://liqul.github.io/blog/assets/rotation.pdf). In this case, we can use the interaction matrix $\Gamma$ to write

$$
\frac{d}{dt}
\begin{pmatrix}
\phi \\ \theta \\ \psi
\end{pmatrix}
=
\begin{pmatrix}
1 & \sin\phi\tan\theta & \cos\phi\tan\theta \\
0 & \cos\phi & -\sin\phi \\
0 & \sin\phi\sec\theta & \cos\phi\sec\theta
\end{pmatrix}
\begin{pmatrix}
p \\ q \\ r
\end{pmatrix}
=
\Gamma \boldsymbol{\omega}.
$$

We can further simplify these kinematics by assuming small angles about level. Focusing only on roll $\phi$ and pitch $\theta$, we have

$$
\begin{align}
\dot{\phi} &\approx p \\
\dot{\theta} &\approx q
\end{align}
$$

We will use these simplified equations for this lab.

---

## Problem Statement

The ultimate goal of this lab and the following one is to design a Kalman filter that fuses gyro and accelerometer measurements and provides an estimate of the tilt angle of the IMU. In this lab, we will focus mainly on the derivation of the state and measurement equations. In the following lab you will work on the design and implementation of the Kalman filter.  Using the approximate continuous-time attitude kinematics using Euler angles as discussed above, we can write

$$
\begin{align}
\dot{\mathbf{x}} &= A\mathbf{x} + B\mathbf{u} + \mathbf{v} \\
\mathbf{y} &= C\mathbf{x} + \mathbf{w}
\end{align}
$$

where $\mathbf{x}:=\left[\begin{array}{c}
\phi\\
b_{p}\\
\theta\\
b_{q}
\end{array}\right]$, $\mathbf{u}:=\left[\begin{array}{c}
y_{p}\\
y_{q}
\end{array}\right]$, $\mathbf{y}:=\left[\begin{array}{c}
y_{\phi}\\
y_{\theta}
\end{array}\right]$, $\mathbb{E}[v(t)]=\mathbb{E}[w(t)]=0$, $\mathbb{E}[v(t)v(t+\tau)] = Q_c\delta(\tau)$ and $\mathbb{E}[w(t)w(t+\tau)] = R_c\delta(\tau)$. Note that we have not supplied any statistics for $\mathbf{v}$ or $\mathbf{w}$. In this case, we will use $Q$ and $R$ in the discrete Kalman filter simply as tuning knobs. 

1. Obtain the matrices $A$, $B$, and $C$ of the continuous-time system above
1. Letting $dt$ denote the discretization period, obtain the matrices $A_d$, $B_d$, and $C_d$ (resulting from the discretization of the continuous-time system above).
1. Given the discrete system model, do you expect a the Kalman filter error covariance to converge to a steady-state solution? Discuss why or why not.